In [ ]:
import requests #Получение данных
import json 
import csv #Запись данных в файл
import pandas as pd
import numpy as np
import re
import os #работа с файлами, папками
import folium #для построения карты

In [ ]:
# Загружаем ключ доступа к API Поиска по организациям
f = open("api_key.txt", "r")
key = f.read()

In [ ]:
# Загружаем информацию о новых жилых комплексах (для ее расширения значениями координат)
df_all_complexes = pd.read_csv("complexes_data.csv", sep=',', index_col='Unnamed: 0')
df_all_complexes

In [ ]:
# Функция coordinates_parser используется для того, чтобы найти координаты жк (поскольку на ЦИАН нет данной информации, а она потребуется при дальнейшем анализе)
# Параметры функции:
# df_houses - датафрейм, где название жк записано в столбце с именем complex_name
# key - ключ доступа к API Поиска по организациям
def coordinates_parser(df_houses,key):
    names_yandex = []
    lat = []
    long = []
    # делаем цикл по названиям комплексов
    for i in list(df_houses.complex_name):
        # Заменяем '&' в названиях жк,так как он используется в качестве разделителя параметров при запросе
        name = i.replace('&','and')
        try:
            # У некоторых жк в скобках перевод названия на русский (берем его, так как яндекс преимущество использует русский в названиях)
            name = name.split('(')[1].split(')')[0]
        except:
            None
        # формируем ссылку для запроса, передаем название жк и ключ к API
        url="https://search-maps.yandex.ru/v1/?text=жк %s&lang=ru_RU&ll=30.3077291, 59.936431&spn=0.7,0.70&results=500&apikey=%s"\
        % (name, key)
        # делаем запрос и сохраняем ответ в data
        r = requests.get(url)
        decoded_data=r.text.encode().decode('utf-8-sig')
        data = json.loads(decoded_data)
        # считаем количество объектов в ответе
        feature_num = len(data['features'])
        
        # Запускаем цикл по объектам. Ищем тот объект, где название категории 'Жилой комплекс'
        it = 0
        # Метка ACTIVE используется для того, чтобы понять нашли мы в результате соответсвующий объект на карте или нет
        mark = 'ACTIVE'

        while it<feature_num and mark =='ACTIVE':     
            if data['features'][it]['properties']['CompanyMetaData']['Categories'][0]['name'] == 'Жилой комплекс':
                #Берем названия объекта на Яндекс картах и его координаты
                names_yandex.append(data['features'][it]['properties']['name'])
                lat.append(data['features'][it]['geometry']['coordinates'][1])
                long.append(data['features'][it]['geometry']['coordinates'][0])
                mark = 'DONE'
            it=it+1
        # Если после прохождения по всем объектам статус искомого объекта ACTIVE - пишем 'None', то есть объект не был найдет на яндекс картах
        if mark=='ACTIVE':
            names_yandex.append('None')
            lat.append('None')
            long.append('None')
    # Добавляем к исходному датафрейму название с Яндекс карт и координаты жк
    df_houses['name_yandex'] = names_yandex
    df_houses['longitude'] = long
    df_houses['latitude'] = lat
    
    return df_houses

In [ ]:
# Расширяем информацию о новых жилых комплексах их географическими координатами
houses_cian_with_coordinate = coordinates_parser(df_all_complexes, key)

In [ ]:
# Сохраняем собранную информацию - она будет позже загружена в ClickHouse
houses_cian_with_coordinate.to_csv('complexes_info.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
# Функция infrastructure_parser используется для сбора данных об инфраструктуре определенного типа вокруг всех ЖК

# Параметры функции:
# df_houses - датафрейм с координатами ЖК в виде longitude и latitude
# word_for_searching - тип искомой инфраструктуры (н-р школа, супермаркет, парк)
# key - ключ доступа к API Поиска по организациям
def infrastructure_parser(df_houses, key, word_for_searching):
    # Цикл по всем ЖК, используя их координаты
    for g in list(zip(df_houses['longitude'],df_houses['latitude'])):
        
        # формируем ссылку для запроса, передаем тип искомого объекта, географические координаты центра зоны поиска (жк) и ключ доступа к API
        url="https://search-maps.yandex.ru/v1/?text=%s&lang=ru_RU&ll=%f,%f&spn=0.03,0.020&results=500&apikey=%s" \
        %(word_for_searching, g[0],g[1],key)
        #делаем запрос и сохраняем ответ в data_js
        r = requests.get(url)
        decoded_data=r.text.encode().decode('utf-8-sig')
        data_js = json.loads(decoded_data)
        
        # Запускаем цикл по всем объектам в ответе и берем данные, которые нам нужны
        for i in range(len(data_js['features'])):
            longitude = data_js['features'][i]['geometry']['coordinates'][0]
            latitude = data_js['features'][i]['geometry']['coordinates'][1]
            name = data_js['features'][i]['properties']['CompanyMetaData']['name']
            address = data_js['features'][i]['properties']['CompanyMetaData']['address']
            com_id = data_js['features'][i]['properties']['CompanyMetaData']['id']
            # Объекту может присваиваться несколько категорий, поэтому мы выделяем главную категорию (стоит первой) и второстепенные
            class_count = len(data_js['features'][i]['properties']['CompanyMetaData']['Categories'])
            extra_categories = None
            try:
                # class - более общая категория с английским названием
                main_class = data_js['features'][i]['properties']['CompanyMetaData']['Categories'][0]['class']
                main_category = data_js['features'][i]['properties']['CompanyMetaData']['Categories'][0]['name']
                
                # Берем максимум 2 дополнительных категории, поэтому если суммарное число категорий больше 3, приравниваем его к 3
                if class_count>3:
                    class_count=3

                for k in range(1,class_count):
                    extra_categories = []
                    try:
                        category_name = data_js['features'][i]['properties']['CompanyMetaData']['Categories'][k]['name']
                        extra_categories.append(category_name)
                    except:
                        None
                # Записываем результат построчно в csv файл в папку infrastructure
                with open(r"infrastructure/loaded_data %s.csv" %word_for_searching, "a", newline="", encoding='utf-8') as file: #'a' - запись строки
                    row = [com_id,longitude,latitude, name,address,main_class, main_category,extra_categories] #строка, которую будем записывать
                    writer = csv.writer(file)
                    writer.writerow(row)
            except:
                None

In [ ]:
# Категории по которым может осуществляться поиск (названия соответсвуют категориям с Яндекс карт)
top_categories = ['Фитнес-клуб', 'Бассейн','Спортплощадка, воркаут','Школа танцев','Гимназия', 'Лицей', 'Общеобразовательная школа',
 'Парк культуры и отдыха', 'Продуктовый гипермаркет', 'Супермаркет', 'Остановка общественного транспорта', 'Железнодорожная станция',
 'Железнодорожный вокзал', 'Станция метро', 'Аптека', 'Больница для взрослых', 'Ресторан', 'Кафе', 'Быстрое питание', 'Пиццерия',
 'Булочная, пекарня', 'Кофейня', 'Кондитерская', 'Булочная', 'Товары для дома', 'Магазин парфюмерии и косметики',
 'Магазин хозтоваров и бытовой химии']

In [ ]:
# Словарь для пользовательских категорий (если необходимо обобщить категории, присваемые Яндексом)
new_category_dict = {
 'фитнес':['Фитнес-клуб', 'Бассейн','Спортплощадка, воркаут','Школа танцев'],
 'школа': ['Гимназия', 'Лицей','Общеобразовательная школа'],
 'парк': ['Парк культуры и отдыха'],
 'супгип': ['Продуктовый гипермаркет','Супермаркет'],
 'транспорт': ['Остановка общественного транспорта','Железнодорожная станция','Железнодорожный вокзал'],
 'метро': ['Станция метро'],
 'аптека':['Аптека'],
 'больница':['Больница для взрослых'],
 'кафе и ресторан': ['Ресторан', 'Кафе', 'Быстрое питание', 'Пиццерия'],
 'пекарня':['Булочная, пекарня','Кофейня','Кондитерская','Булочная'],
 'дрогери': ['Товары для дома','Магазин парфюмерии и косметики','Магазин хозтоваров и бытовой химии']}

In [ ]:
# Данная функция запускает работу функции infrastructure_parser по категориям из списка category_list,
# собирает все файлы из папки infrastructure, объединяет разультаты в общий df и добавляет пользовательские категории

# Параметры функции:
# df_houses - датафрейм с координатами ЖК в виде longitude и latitude
# category_list - список типов искомой инфраструктуры (н-р школа, супермаркет, парк)
# category_dict - словарь с пользовательскими категориями
# key - ключ доступа к API Поиска по организациям
def loading_and_combining_infr(df_houses, category_list,category_dict,key):
    #Создаем датафрейм, куда будем сохранять всю инфраструктуру
    infr_all = pd.DataFrame(columns=['com_id', 'longitude', 'latitude', 'name', 'address', 'main_class',
       'main_category', 'extra_categories'])
    # запускаем цикл парсинга по всем типам инфраструктуры
    for i in category_list:
        infrastructure_parser(df_houses, key, i)
    # Собираем файлы, которые были загружены в папку infrastructure
    files_infr = os.listdir('/infrastructure')
    # Цикл по всем файлам
    for j in files_infr:
        # Сохраняем файл в df
        infr = pd.read_csv('infrastructure/%s' %j,names=['com_id', 'longitude', 'latitude', 'name', 'address', 'main_class',
       'main_category', 'extra_categories'])
        # Добавляем к общей таблице
        infr_all = infr_all.append(infr)
    # Удаляем дубликаты, так как районы поиска пересекаются и объекты могут быть загружены несколько раз
    infr_all = infr_all.drop_duplicates(subset='com_id')
    
    # Добавляем пользовательскую категорию из словаря
    our_category_list = []
    for i in list(infr_all.main_category):
        for cat in category_dict.keys():
            if i in category_dict[cat]:
                our_category_list.append(cat)
    infr_all['our_category'] = our_category_list
    infr_all = infr_all[['com_id', 'longitude', 'latitude', 'name', 'address', 'main_class',
       'main_category','our_category', 'extra_categories']]
    return infr_all

In [ ]:
# Запускаем функцию для загрузки данных инфраструктуры 
infrastructure_data_for_loading = loading_and_combining_infr(df_all_complexes, top_categories, new_category_dict, key)

In [ ]:
# Сохраняем собранную информацию - она будет позже загружена в ClickHouse
infrastructure_data_for_loading.to_csv('infrastructure_processed.csv', sep=';', encoding='utf-8', index=False)